In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score

In [2]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [4]:
# Z 'PassengerID' wyciągam numer grupy pasażera do osobnej kolumny

df['PassengerGroup'] = df['PassengerId'].str.split('_').str[0]

In [5]:
passenger_group_val_counts = df['PassengerGroup'].value_counts()

In [6]:
# Z 'PassengerID' wyciągam liczebność grupy pasażera i zapisuję ją
# w osobnej kolumnie

df['GroupSize'] = df['PassengerGroup'].map(passenger_group_val_counts)

In [7]:
# Zamieniam indexy na 'PassengerId'

df.index = df.PassengerId

In [8]:
# Wyrzucam kolumnę 'PassengerId'
df.drop('PassengerId', axis=1, inplace=True)

In [9]:
df

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,PassengerGroup,GroupSize
PassengerId,,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,1
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,1
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,2
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,2
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,9276,1
9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,9278,1
9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,9279,1


In [10]:
# Wyciągam informacje o tym gdzie pasażer przebywał z 'Cabin' 
# do trzech osobnych kolumn i zrzucam kolumne

df['CabinDeck'] = df['Cabin'].str.split('/').str[0]
df['CabinNum'] = df['Cabin'].str.split('/').str[1]
df['CabinSide'] = df['Cabin'].str.split('/').str[2]

In [11]:
df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,PassengerGroup,GroupSize,CabinDeck,CabinNum,CabinSide
PassengerId,,,,,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,1,B,0,P
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,1,F,0,S
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,2,A,0,S
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,2,A,0,S
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,1,F,1,S


In [12]:
df.drop('Cabin', axis=1, inplace=True)
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,PassengerGroup,GroupSize,CabinDeck,CabinNum,CabinSide
PassengerId,,,,,,,,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,1,B,0,P
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,1,F,0,S
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,2,A,0,S
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,2,A,0,S
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0004,1,F,1,S


In [13]:
# Wyrzucam kolumne z imieniem i nazwiskiem ze względu na zbyt dużą różnorodność

df.drop("Name", axis=1, inplace=True)
df

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,PassengerGroup,GroupSize,CabinDeck,CabinNum,CabinSide
PassengerId,,,,,,,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0001,1,B,0,P
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0002,1,F,0,S
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0003,2,A,0,S
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0003,2,A,0,S
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,0004,1,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,9276,1,A,98,P
9278_01,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,9278,1,G,1499,S
9279_01,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,9279,1,G,1500,S


In [14]:
for col in df.columns:
    print(col, df[col].value_counts(dropna=False))

HomePlanet HomePlanet
Earth     4602
Europa    2131
Mars      1759
NaN        201
Name: count, dtype: int64
CryoSleep CryoSleep
False    5439
True     3037
NaN       217
Name: count, dtype: int64
Destination Destination
TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
NaN               182
Name: count, dtype: int64
Age Age
24.0    324
18.0    320
21.0    311
19.0    293
23.0    292
       ... 
72.0      4
78.0      3
79.0      3
76.0      2
77.0      2
Name: count, Length: 81, dtype: int64
VIP VIP
False    8291
NaN       203
True      199
Name: count, dtype: int64
RoomService RoomService
0.0       5577
NaN        181
1.0        117
2.0         79
3.0         61
          ... 
1612.0       1
2598.0       1
632.0        1
378.0        1
745.0        1
Name: count, Length: 1274, dtype: int64
FoodCourt FoodCourt
0.0       5456
NaN        183
1.0        116
2.0         75
3.0         53
          ... 
3846.0       1
5193.0       1
312.0        1
827.0        1
4688.0       

In [15]:
for col in df.columns:
    print(col, len(df[col].unique()))

HomePlanet 4
CryoSleep 3
Destination 4
Age 81
VIP 3
RoomService 1274
FoodCourt 1508
ShoppingMall 1116
Spa 1328
VRDeck 1307
Transported 2
PassengerGroup 6217
GroupSize 8
CabinDeck 9
CabinNum 1818
CabinSide 3


In [16]:
# Usuwam 'CabinNum' z powodu na liczebność jego unikatowych wyników

df.drop('CabinNum', axis=1, inplace=True)

In [17]:
# Usuwam 'PassengerGroup' z powodu na liczebność jego unikatowych wyników

df.drop('PassengerGroup', axis=1, inplace=True)

In [18]:
# Wyznaczam target

y = df.Transported

In [19]:
y

PassengerId
0001_01    False
0002_01     True
0003_01    False
0003_02    False
0004_01     True
           ...  
9276_01    False
9278_01    False
9279_01     True
9280_01    False
9280_02     True
Name: Transported, Length: 8693, dtype: bool

In [20]:
# Dziele kolumny na kategoryczne i numeryczne

num_columns = df.select_dtypes(['int', 'float'])
cat_columns = df.select_dtypes(['object'])

In [21]:
num_columns

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,GroupSize
PassengerId,,,,,,,
0001_01,39.0,0.0,0.0,0.0,0.0,0.0,1
0002_01,24.0,109.0,9.0,25.0,549.0,44.0,1
0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,2
0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,2
0004_01,16.0,303.0,70.0,151.0,565.0,2.0,1
...,...,...,...,...,...,...,...
9276_01,41.0,0.0,6819.0,0.0,1643.0,74.0,1
9278_01,18.0,0.0,0.0,0.0,0.0,0.0,1
9279_01,26.0,0.0,0.0,1872.0,1.0,0.0,1


In [22]:
cat_columns

,HomePlanet,CryoSleep,Destination,VIP,CabinDeck,CabinSide
PassengerId,,,,,,
0001_01,Europa,False,TRAPPIST-1e,False,B,P
0002_01,Earth,False,TRAPPIST-1e,False,F,S
0003_01,Europa,False,TRAPPIST-1e,True,A,S
0003_02,Europa,False,TRAPPIST-1e,False,A,S
0004_01,Earth,False,TRAPPIST-1e,False,F,S
...,...,...,...,...,...,...
9276_01,Europa,False,55 Cancri e,True,A,P
9278_01,Earth,True,PSO J318.5-22,False,G,S
9279_01,Earth,False,TRAPPIST-1e,False,G,S


### Missing values

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Destination   8511 non-null   object 
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   object 
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  Transported   8693 non-null   bool   
 11  GroupSize     8693 non-null   int64  
 12  CabinDeck     8494 non-null   object 
 13  CabinSide     8494 non-null   object 
dtypes: bool(1), float64(6), int64(1), object(6)
memory usage: 1.2+ MB


In [24]:
num_columns.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           8514 non-null   float64
 1   RoomService   8512 non-null   float64
 2   FoodCourt     8510 non-null   float64
 3   ShoppingMall  8485 non-null   float64
 4   Spa           8510 non-null   float64
 5   VRDeck        8505 non-null   float64
 6   GroupSize     8693 non-null   int64  
dtypes: float64(6), int64(1)
memory usage: 801.4+ KB


In [25]:
# Uzupełniam kolumny numeryczne najczęściej występującą wartością

filled_num_columns = num_columns.fillna(num_columns.mode().iloc[0])

In [26]:
filled_num_columns.isna().sum()

Age             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
GroupSize       0
dtype: int64

In [27]:
# Wykonuje OneHotEncoding na kolumnach kategorycznych

ohe = OneHotEncoder()

filled_cat_columns = pd.DataFrame(ohe.fit_transform(cat_columns).toarray(), index=cat_columns.index, columns=ohe.get_feature_names_out())
filled_cat_columns

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_nan,CryoSleep_False,CryoSleep_True,CryoSleep_nan,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,...,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T,CabinDeck_nan,CabinSide_P,CabinSide_S,CabinSide_nan
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0002_01,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
0003_01,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0003_02,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0004_01,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9278_01,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9279_01,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [28]:
# Łącze oba zbiary kolumn w X

X = pd.concat([filled_num_columns, filled_cat_columns], axis=1)
X

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,GroupSize,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,...,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T,CabinDeck_nan,CabinSide_P,CabinSide_S,CabinSide_nan
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,39.0,0.0,0.0,0.0,0.0,0.0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0002_01,24.0,109.0,9.0,25.0,549.0,44.0,1,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,2,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,2,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0004_01,16.0,303.0,70.0,151.0,565.0,2.0,1,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,41.0,0.0,6819.0,0.0,1643.0,74.0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9278_01,18.0,0.0,0.0,0.0,0.0,0.0,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
9279_01,26.0,0.0,0.0,1872.0,1.0,0.0,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [29]:
X.isna().sum()

Age                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
GroupSize                    0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
HomePlanet_nan               0
CryoSleep_False              0
CryoSleep_True               0
CryoSleep_nan                0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
Destination_nan              0
VIP_False                    0
VIP_True                     0
VIP_nan                      0
CabinDeck_A                  0
CabinDeck_B                  0
CabinDeck_C                  0
CabinDeck_D                  0
CabinDeck_E                  0
CabinDeck_F                  0
CabinDeck_G                  0
CabinDeck_T                  0
CabinDeck_nan                0
CabinSide_P                  0
CabinSide_S                  0
CabinSid

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Random Forest Classifier

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

param_grid = {
    'n_estimators': [150, 200, 250],  # Number of trees in the forest       # Maximum depth of the trees
    'min_samples_split': [1, 2],   # Minimum number of samples required to split a node
    'min_samples_leaf': [4, 6, 8]      # Minimum number of samples required at each leaf node
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=1), param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
45 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/home/kuba/an

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
             param_grid={'min_samples_leaf': [4, 6, 8],
                         'min_samples_split': [1, 2],
                         'n_estimators': [150, 200, 250]},
             scoring='accuracy')

In [31]:
y_pred = grid_search.predict(X_test)

In [32]:
accuracy_score(y_test, y_pred)

0.7993099482461185

In [33]:
grid_search.best_params_

{'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 150}

### Naiwny Krasyfikator Bayesa

In [34]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB 

In [35]:
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7809085681426107

In [40]:
model = MultinomialNB()
param_grid = {
    'alpha': [0.1, 0.5, 1.0],
    'fit_prior': [True, False]
}
gs = GridSearchCV(model, param_grid, cv=10)
gs.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=MultinomialNB(),
             param_grid={'alpha': [0.1, 0.5, 1.0], 'fit_prior': [True, False]})

In [42]:
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)

0.7809085681426107

In [36]:
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7745830937320299

In [37]:
model = BernoulliNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7303047728579644

In [44]:
model = BernoulliNB()
param_grid = {
    'alpha': [0.1, 0.5, 1.0],  
    'binarize': [None, 0.0, 0.5]
}
gs = GridSearchCV(model, param_grid, cv=10)
gs.fit(X_train, y_train)

/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:1231: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:1231: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:1231: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:1231: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:1231: RuntimeWarning: invalid value encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
/home/kuba/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:1231: R

GridSearchCV(cv=10, estimator=BernoulliNB(),
             param_grid={'alpha': [0.1, 0.5, 1.0],
                         'binarize': [None, 0.0, 0.5]})

In [45]:
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)

0.7303047728579644

### Support Vector Machine

In [32]:
from sklearn.svm import SVC

In [45]:
model = SVC()
param_grid = {
    'C': [10, 50, 100],
    'random_state': [1]
}
gs = GridSearchCV(model, param_grid, cv=10)
gs.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [10, 50, 100], 'random_state': [1]})

In [38]:
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)

0.8033352501437608

In [39]:
gs.best_params_

{'C': 10}

In [46]:
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)

0.7975848188614146

In [47]:
gs.best_params_

{'C': 50, 'random_state': 1}

### K Nearest Neighbours

In [48]:
from sklearn.neighbors import KNeighborsClassifier

In [58]:
model = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [11, 15, 20, 25],  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Weight function used in prediction
    'metric': ['euclidean', 'manhattan'] 
}
gs = GridSearchCV(model, param_grid, cv=10)
gs.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [11, 15, 20, 25],
                         'weights': ['uniform', 'distance']})

In [53]:
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)

0.7878090856814262

In [54]:
gs.best_params_

{'n_neighbors': 11}

In [56]:
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)

0.7918343875790684

In [57]:
gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'uniform'}

In [59]:
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)

0.7952846463484762

In [60]:
gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 20, 'weights': 'uniform'}

### Gradient Boosting Classifier

In [61]:
from sklearn.ensemble import GradientBoostingClassifier

In [62]:
model = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01, max_features='sqrt', max_depth=2,
                                  random_state=0, loss='exponential')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

0.7993099482461185

In [67]:
model = GradientBoostingClassifier()

params = {
    'n_estimators': [50, 100, 150],  # Number of boosting stages
    'learning_rate': [0.05, 0.1, 0.2],  # Learning rate shrinks the contribution of each tree
    'max_depth': [3, 5, 7],  # Maximum depth of the individual trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

gs = GridSearchCV(model, params)
gs.fit(X_train, y_train)

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]})

In [68]:
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)

0.8004600345025877

In [69]:
gs.best_params_

{'learning_rate': 0.1,
 'max_depth': 5,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 100}